In [12]:
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Generate data
d1 = np.random.normal(loc=0.0, scale=1.0, size=3500)
d2 = np.random.normal(loc=0.3, scale=0.2, size=2100)

# Create a DataFrame for Plotly
df = pd.DataFrame({
    "Value": np.concatenate([d1, d2]),
    "Dataset": ["d1"] * len(d1) + ["d2"] * len(d2)
})

# Create histogram
fig = px.histogram(df, x="Value", color="Dataset", nbins=20, opacity=0.7, barmode="overlay")

# Adjust layout
fig.update_layout(
    title='Histogram of Relative Price Differences Between Two Stores',
    xaxis_title='Относительная разница стоимости',
    yaxis_title='Count',
    bargap=0.1
)

# Show plot
plotly.offline.plot(fig)

'temp-plot.html'

In [14]:
# Generate data
d1 = np.random.normal(loc=0.0, scale=1.0, size=3500)
d2 = np.random.normal(loc=0.3, scale=0.2, size=2100)

# Create histogram
fig = go.Figure()

# Add histogram for d1
fig.add_trace(go.Histogram(
    x=d1,
    name="d1",
    opacity=0.7,
    xbins=dict(start=0, end=0.6, size=0.1)  # Fixed bins
))

# Add histogram for d2
fig.add_trace(go.Histogram(
    x=d2,
    name="d2",
    opacity=0.7,
    xbins=dict(start=-0.6, end=0.6, size=0.1)
))

# Adjust layout
fig.update_layout(
    title='Histogram of Relative Price Differences Between Two Stores',
    xaxis_title='Относительная разница стоимости',
    yaxis_title='Count',
    bargap=0.1,
    barmode="overlay"  # Overlay histograms
)

# Show plot
plotly.offline.plot(fig)


'temp-plot.html'

In [28]:
from sqlalchemy import select
from sqlalchemy.orm import Session, aliased
from db_populate.session import db_session_as_kwarg
from db_populate.models import Match as SaMatch
from db_populate.models import Product as SaProduct


@db_session_as_kwarg
def get_next_product(session: Session) -> list[list]:
    p1_alias = aliased(SaProduct)
    p2_alias = aliased(SaProduct)

    query = session.execute(
        select(p1_alias.title, p1_alias.price, p2_alias.price).select_from(SaMatch)
        .join(p1_alias, SaMatch.product_1_id == p1_alias.id)
        .join(p2_alias, SaMatch.product_2_id == p2_alias.id)
    )

    rows = []
    for sa_row in query.all():
        r = sa_row
        rows.append([
            sa_row[0],
            sa_row[1],
            sa_row[2],
        ])

    return rows

rows = get_next_product()

In [29]:
rows

[['Профиль угловой сетчатый 35х35x3000 мм, оцинкованный', 77.0, 89.0],
 ['Профиль угловой 25х25х3000 мм, оцинкованный 0,25 мм', 96.0, 97.0],
 ['Профиль угловой с армирующей сеткой 100х150x2500 мм, ПВХ', 108.0, 135.0],
 ['Веревка джутовая 6 мм 20 м', 237.0, 78.0],
 ['Вороток для сваи 108 мм НСЗ', 749.0, 949.0],
 ['Блок газобетонный ВКБлок D500 625х300х250 мм', 6.0, 639.0],
 ['Блок газобетонный ВКБлок D500 625х100х250 мм', 6.0, 142.0],
 ['Блок газобетонный Bonolit D500 600х100х250 мм', 6.0, 128.0],
 ['Блок газобетонный Bonolit D500 600x75x250 мм', 6.0, 96.0],
 ['Блок газобетонный Bonolit D500 600х50х250 мм', 7.0, 79.0],
 ['Блок газобетонный ЕЗСМ D500 625x100х250 мм', 5.0, 142.0],
 ['Блок газобетонный ЕЗСМ D500 625x200х250 мм', 5.0, 289.0],
 ['Блок газобетонный Bonolit D500 600х250х200 мм', 6.0, 259.0],
 ['Блок газобетонный Bonolit D600 600х100х250 мм', 6.0, 128.0],
 ['Фиксатор Звездочка з/с 20 мм; 25 мм', 1.0, 209.0],
 ['Плита гипсовая пазогребневая Волма пустотелая 667х500х80 мм', 294.0

In [36]:
real_df = pd.DataFrame(rows, columns=["title", "price_1", "price_2"])

real_df["delta"] = real_df["price_1"] - real_df["price_2"]
real_df["relative_delta"] = real_df["delta"] / real_df["price_2"]
real_df

,title,price_1,price_2,delta,relative_delta
0,"Профиль угловой сетчатый 35х35x3000 мм, оцинко...",77.0,89.0,-12.0,-0.134831
1,"Профиль угловой 25х25х3000 мм, оцинкованный 0,...",96.0,97.0,-1.0,-0.010309
2,Профиль угловой с армирующей сеткой 100х150x25...,108.0,135.0,-27.0,-0.200000
3,Веревка джутовая 6 мм 20 м,237.0,78.0,159.0,2.038462
4,Вороток для сваи 108 мм НСЗ,749.0,949.0,-200.0,-0.210748
...,...,...,...,...,...
102,Затирка эпоксидная Litokol Starlike Evo S.202 ...,4.0,5099.0,-5095.0,-0.999216
103,Колодка клеммная 12 пар 1-4 мм²,99.0,100.0,-1.0,-0.010000
104,Кабель-канал белый 16x16 2 м T.Plast,89.0,81.0,8.0,0.098765
105,Труба гофрированная ПНД черная 16 мм с протяжк...,1.0,1149.0,-1148.0,-0.999130


In [43]:
# Create histogram
fig = go.Figure()

# Add histogram for d1
fig.add_trace(go.Histogram(
    x=real_df["relative_delta"],
    name="В OBI дешевле",
    opacity=1,
    marker_color="#E45756",
    xbins=dict(start=0, end=0.9, size=0.1)  # Fixed bins
))
# Add histogram for d1
fig.add_trace(go.Histogram(
    x=real_df["relative_delta"],
    name="В Строительный двор дешевле",
    opacity=1,
    marker_color="#54A24B",
    xbins=dict(start=-0.9, end=0, size=0.1)  # Fixed bins
))

# Adjust layout
fig.update_layout(
    title='Гистограмма распределения разницы цен Строительный двор - OBI',
    xaxis_title='Относительная разница стоимости',
    yaxis_title='Кол-во',
    bargap=0.1,
    # barmode="overlay"  # Overlay histograms
)

# Show plot
plotly.offline.plot(fig)

'temp-plot.html'